In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage  # for hierarchical clustering

## K-means clustering

http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

#### First let's create some random data:

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_features=2, centers=2,n_samples=20,random_state=9)

# display
plt.scatter(X[:,0],X[:,1],c='r')
plt.show()

In [ ]:
# Let's see what running k-means on this data would look like, step by step:

# arbitrary assignment of centers
cent=np.asarray([[-6,-6],[-2,-1]])
plt.figure(figsize=(12,3))

# first of four panels just has the original data
plt.subplot(1,4,1)
plt.scatter(X[:,0],X[:,1],c="r")

# for each k-means step
for i in range(3):
    
    # show on a separate subplot
    plt.subplot(1,4,i+2)
    
    # compute euclidean distance to each cluster center for each point
    distance_to_cluster_0 = np.sum((X-cent[0,:])**2,axis=1)**0.5
    distance_to_cluster_1 = np.sum((X-cent[1,:])**2,axis=1)**0.5
    
    # compute cluster assignment for each point
    y_t = (distance_to_cluster_0 > distance_to_cluster_1) 
    
    # plot points and centers
    plt.scatter(X[:,0],X[:,1],c=y_t,cmap=plt.cm.cool)
    plt.scatter(cent[0,0],cent[0,1],c="b",s=100)
    plt.scatter(cent[1,0],cent[1,1],c="r",s=100)

    # recompute centers
    cent=np.asarray([[np.sum(X[:,0]*(1-y_t))/(np.sum(1-y_t)),np.sum(X[:,1]*(1-y_t))/np.sum(1-y_t)],
            [np.sum(X[:,0]*(y_t))/np.sum(y_t),np.sum(X[:,1]*(y_t))/np.sum(y_t)]])

# show plots
plt.show()

In [ ]:
# Use k-means package from sklearn
from sklearn.cluster import KMeans
km = KMeans(random_state=234,n_clusters=2)
res=km.fit(X)

print 'Cluster assignments:',res.labels_
print '\nCluster centers:\n',res.cluster_centers_
print '\nSum of squared errors:',res.inertia_

# plot points and cluster assigments
plt.figure(figsize=(3,3))
plt.scatter(X[:,0],X[:,1],c=res.labels_,cmap=plt.cm.cool)
plt.scatter(res.cluster_centers_[0,0],res.cluster_centers_[0,1],c="b",s=100)
plt.scatter(res.cluster_centers_[1,0],res.cluster_centers_[1,1],c="r",s=100)
plt.show()

## Gaussian Mixture Models (EM)

http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

In [ ]:
# Use Gaussian mixture models (EM) package from sklearn
from sklearn.mixture import GaussianMixture
from scipy import linalg
import matplotlib as mpl
gmm = GaussianMixture(n_components=2,random_state=100)
res = gmm.fit(X)

print 'Probabilities of belonging to cluster 1:\n',res.predict_proba(X)[:,1]
print '\nCluster centers:\n',res.means_
print '\nCluster covariances:\n',res.covariances_
print '\nLog-likelihood per sample:',res.score(X)

plt.figure(figsize=(3,3))
splot = plt.subplot(111)

# plot ellipses using mean and covariance matrix for each cluster
colors = ['blue','red']
for i in range(2):
    mean = res.means_[i]
    var = res.covariances_[i]
    v, w = linalg.eigh(var)
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180. * angle / np.pi  # convert to degrees
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=colors[i])
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

# plot points and cluster assigments
plt.scatter(X[:,0],X[:,1],c=res.predict_proba(X)[:,1],cmap=plt.cm.cool)
plt.show()

### Practice #1.  Using 311 complaints data set to cluster and visualize NYC census tract areas.

In [ ]:
import pandas as pd
from sklearn import preprocessing

data311=pd.read_csv("311DataForClustering.csv")
test=preprocessing.normalize(data311.iloc[:,1:])
test.shape

In [ ]:
# Visualization: example with random cluster assignments
random_clusters = np.random.randint(4,size=len(data311.index))
res=pd.concat([pd.Series(data311.geoid),pd.Series(random_clusters,name='cluster')],axis=1)
res.head()

Reference for this function: 311 Project (Lingjing Wang, Cheng Qian, Constantine Kontokosta, Stanislav Sobolevsky)

https://arxiv.org/pdf/1611.06660.pdf

pip install pyshp

In [ ]:
def visualization(result):
    import shapefile
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from matplotlib.patches import Polygon
    from matplotlib.collections import PatchCollection
    #   -- input --
    Boston_geoid=list(data311.geoid.unique())
    sf = shapefile.Reader("nyct2010_16b/nyct2010.shp")
    recs    = sf.records()
    test=pd.DataFrame(recs)
    test.loc[:,'county']=0
    test.loc[test.iloc[:,2]=="Staten Island","county"]="085"
    test.loc[test.iloc[:,2]=="Manhattan","county"]="061"
    test.loc[test.iloc[:,2]=="Brooklyn","county"]="047"
    test.loc[test.iloc[:,2]=="Bronx","county"]="005"
    test.loc[test.iloc[:,2]=="Queens","county"]="081"
    test.loc[:,"geoid"]=0
    test.loc[:,"geoid"]="36"+test.county+test.iloc[:,3].apply(str)

    shapes  = sf.shapes()
    Nshp    = len(shapes)
    cns     = []
    for nshp in xrange(Nshp):
        cns.append(recs[nshp][1])
    cns = np.array(cns)

    cmap = plt.cm.Spectral(np.linspace(0,1,max(result.iloc[:,-1])+1))

    fig=plt.figure(figsize = (10,10)) 
    fig.add_subplot(111)
    ax = fig.gca() 
    for nshp in xrange(Nshp):
        if int(test.iloc[nshp,-1]) in Boston_geoid:
            k=result[result.geoid==int(test.iloc[nshp,-1])].iloc[0,-1]
            c=cmap[k][0:3]  
            ptchs   = []
            pts     = np.array(shapes[nshp].points)
            prt     = shapes[nshp].parts
            par     = list(prt) + [pts.shape[0]]
            for pij in xrange(len(prt)):
                ptchs.append(Polygon(pts[par[pij]:par[pij+1]]))
            ax.add_collection(PatchCollection(ptchs,facecolor=c,edgecolor='k', linewidths=.5))
        ax.axis('scaled')
    #ax.set_title("NYC Clustering based on 311 service request data")

    import matplotlib.patches as mpatches
    clum_num=len(result.iloc[:,-1].unique())

    handles=[]
    for t in range(clum_num):
        locals()["patch_{}".format(t)] = mpatches.Patch(color=cmap[t][0:3] , label='Cluster'+str(t+1))
        handles.append(locals()["patch_{}".format(t)])
    #plt.axis('off')
    plt.xticks([], [])
    plt.yticks([],[])
    plt.legend(handles=handles,loc='upper left',prop={'size':8})
    plt.show()

In [ ]:
visualization(result=res)

### Now try running k-means with between 2 and 5 clusters.  For each n_clusters, report the impurity and visualize the results.

In [ ]:
# your code here

## Hierarchical Clustering

https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html

https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/

(as an alternative, could use sklearn.cluster.AgglomerativeClustering, but not such great visualization.)


In [ ]:
# same randomly generated points as above
from sklearn.datasets import make_blobs
X, y = make_blobs(n_features=2, centers=2,n_samples=20,random_state=9)

Linkage:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage

An (n−1) by 4 matrix Z is returned. At the i-th iteration, clusters with indices Z[i, 0] and Z[i, 1] are combined to form cluster n+i. A cluster with an index less than nn corresponds to one of the nn original observations. The distance between clusters Z[i, 0] and Z[i, 1] is given by Z[i, 2]. The fourth value Z[i, 3] represents the number of original observations in the newly formed cluster.

### Single Linkage

In [ ]:
Z_single = linkage(X, 'single')
print Z_single

In [ ]:
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z_single,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=12.,  # font size for the x axis labels
)
plt.show()

### Form Clusters:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html#scipy.cluster.hierarchy.fcluster

#### Set number of clusters or threshold distance. 

In [ ]:
from scipy.cluster.hierarchy import fcluster

In [ ]:
# split into three clusters
fcluster(Z_single, 3, criterion='maxclust')

In [ ]:
# Threshold distance = 2.1; break all links longer than the threshold distance.
# Note that higher threshold distance corresponds to fewer clusters.
fcluster(Z_single, t=2.1, criterion='distance')

#### A better dendrogram function (re-used from the blog): 

In [ ]:
def fancy_dendrogram(*args, **kwargs): # This code is resued from the blog.
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata


# t is distance threshold to form number of clusters.  p is number of visualization slots (see example)
def distance_thr(Z,t,p): 
    plt.figure(figsize=(10,6))
    fancy_dendrogram(
        Z,
        truncate_mode='lastp',
        p=p,
        leaf_rotation=90.,
        leaf_font_size=12.,
        show_contracted=True,
        annotate_above=10,
        max_d=t,
    )
    plt.show()

In [ ]:
distance_thr(Z_single,t=2.1,p=20)

In [ ]:
distance_thr(Z_single,t=2.1,p=10)

### Practice #2:

Try 'complete' linkage (and maybe other linkage metrics if you have time), and compare your results with the single linkage. Do these algorithms give you the same results?

In [ ]:
# your code here

### Choosing the number of clusters:

There are numerous criteria for choosing the number of clusters.  See, for example:

https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set

In [ ]:
X, y = make_blobs(n_features=2, centers=3,n_samples=50,random_state=9999)
plt.scatter(X[:,0],X[:,1],c="r")
plt.show()

In [ ]:
Z = linkage(X, 'single')

In [ ]:
distance_thr(Z,t=2.5,p=20)

#### Silhouette Coefficient

One example of a metric used to choose number of clusters:

http://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient

For each sample, compute (b-a)/max(a,b), where:

a: The mean distance between a sample and all other points in the same class.

b: The mean distance between a sample and all other points in the next nearest cluster.

Then take the mean over all samples.

Larger silhouette score (closer to 1) = better clustering.

In [ ]:
from sklearn.metrics import silhouette_score

# Choose a range of cluster numbers that you would like to evalulate:
range_n_clusters = range(2,10)
Z = linkage(X, 'single')

for n_clusters in range_n_clusters:
    cluster_labels=fcluster(Z, n_clusters, criterion='maxclust')    
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

In [ ]:
# As an alternative, look for an elbow in the within-cluster SSE (k-means example):
SSE = []
for i in range(2,10):
    km = KMeans(n_clusters = i)
    res=km.fit(X)
    SSE.append(res.inertia_)
print SSE

plt.gca()
plt.plot(range(2,10),SSE)
plt.xlabel("Clusters")
plt.ylabel("Sum of squared errors")
plt.title("SSE vs. number of clusters")
plt.xlim(1.5,9.5)
plt.show()

## Leader Clustering 

Leader clustering will not be covered in this lab, but if you're interested you can see:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.leaders.html#scipy.cluster.hierarchy.leaders